In [1]:
using ITensors
ITensors.set_warn_order(18);

# DMRG for spinless t-V on Honeycomb lattice

In [2]:
Lx, Ly = 2, 2;
t = 1.0;
nTot = Lx*Ly*2
sites = siteinds("Fermion", Lx*Ly*2; conserve_qns=true);
state = ["Emp" for n in 1:Lx*Ly*2]
for i in 1:Lx*Ly
    state[i] = "Occ" # start from half filled
end

psi0 = randomMPS(sites, state, 10);
nsweeps = 6
maxdim = [50, 100, 200, 400, 800, 800]
cutoff = [1E-12];

function site2Idx(ix, iy, iu)
    return Lx*Ly*(iu) + (ix)*Ly + iy + 1
end;

cb=combiner(dag(sites)...);
cbp=combiner(sites'...);

In [8]:
function eFromV(v, ed=false)
    os = OpSum();
    for ix in 0:Lx-1
        for iy in 0:Ly-1
            os += -t, "Cdag", site2Idx(ix, iy, 0), "C", site2Idx(ix, iy, 1)
            os += -t, "Cdag", site2Idx(ix, iy, 1), "C", site2Idx(ix, iy, 0)
            os += -t, "Cdag", site2Idx(ix, iy, 0), "C", site2Idx((ix+1)%Lx, iy, 1)
            os += -t, "Cdag", site2Idx((ix+1)%Lx, iy, 1), "C", site2Idx(ix, iy, 0)
            os += -t, "Cdag", site2Idx(ix, iy, 0), "C", site2Idx(ix, (iy+1)%Ly, 1)
            os += -t, "Cdag", site2Idx(ix, (iy+1)%Ly, 1), "C", site2Idx(ix, iy, 0)
            os += v, "N", site2Idx(ix, iy, 0), "N", site2Idx(ix, iy, 1)
            os += v, "N", site2Idx(ix, iy, 0), "N", site2Idx((ix+1)%Lx, iy, 1)
            os += v, "N", site2Idx(ix, iy, 0), "N", site2Idx(ix, (iy+1)%Ly, 1)
        end
    end
    H = MPO(os, sites);
    energy, psi = dmrg(H, psi0; nsweeps, maxdim, cutoff, outputlevel = 0);
    d = fill(0.0, Lx*Ly*2)
    for j in 1:Lx*Ly*2
        orthogonalize!(psi, j)
        psidag_j = dag(prime(psi[j], "Site"))
        d[j] = scalar(psidag_j * op(sites, "N", j) * psi[j])
    end
    # println("V=", v, " E=", energy-0.375*nTot*v, "\ndensity=", d)
    println("V=", v, " E=", energy-0.375*nTot*v)

    if ed
        H_ED = prod(H)*cb*cbp
        E,V=eigen(H_ED,combinedind(cb),combinedind(cbp))
        println("E from ED = ", sort(array(real(diag(E))))[1] - 0.375*nTot*v)
    end
end

eFromV (generic function with 2 methods)

In [9]:
eFromV(0.7, true)

V=0.7 E=-6.6456706584805545
E from ED = -6.6456706584805545


In [ ]:
for v in 0.1:0.1:2.0
    eFromV(v)
end

# DMRG for spinless t-V on Square lattice

In [18]:
Lx, Ly = 6, 6;
t = 1.0;
sites = siteinds("Fermion", Lx*Ly; conserve_qns=false);
state = ["Emp" for n in 1:Lx*Ly]
for i in 1:Int(Lx*Ly/2)
    state[i] = "Occ" # start from half filled
end

psi0 = randomMPS(sites, state, 10);
nsweeps = 6
maxdim = [50, 100, 200, 400, 800, 800]
cutoff = [1E-12];

function site2Idx(ix, iy)
    return (ix)*Ly + iy + 1
end;

cb=combiner(dag(sites)...);
cbp=combiner(sites'...);

In [19]:
function eFromV(v, delta, ed=false)
    os = OpSum();
    for ix in 0:Lx-1
        for iy in 0:Ly-1
            i, j1, j2 = site2Idx(ix, iy), site2Idx((ix+1)%Lx, iy), site2Idx(ix, (iy+1)%Ly)
            
            os += -t, "Cdag", i, "C", j1
            os += -t, "Cdag", j1, "C", i
            os += -t, "Cdag", i, "C", j2
            os += -t, "Cdag", j2, "C", i

            os += delta, "Cdag", j1, "Cdag", i
            os += delta, "C", i, "C", j1
            os += 1.0im * delta, "Cdag", j2, "Cdag", i
            os += -1.0im * delta, "C", i, "C", j2

            os += v, "N", i, "N", j1
            os += v, "N", i, "N", j2
            os += (-2.0*v), "N", i
        end
    end
    H = MPO(os, sites);
    energy, psi = dmrg(H, psi0; nsweeps, maxdim, cutoff, outputlevel = 0);
    d = fill(0.0, Lx*Ly*2)
    for j in 1:Lx*Ly
        orthogonalize!(psi, j)
        psidag_j = dag(prime(psi[j], "Site"))
        d[j] = scalar(psidag_j * op(sites, "N", j) * psi[j])
    end
    # println("V=", v, " E=", energy-0.375*nTot*v, "\ndensity=", d)
    println("V=", v, " E=", energy + (0.5 * Lx * Ly * v) )

    if ed
        H_ED = prod(H)*cb*cbp
        E,V=eigen(H_ED,combinedind(cb),combinedind(cbp))
        println("E from ED = ", sort(array(real(diag(E))))[1] + 0.5*Lx*Ly*v)
    end
end

eFromV (generic function with 2 methods)

In [16]:
eFromV(0.7, 0.5), eFromV(0.7, 0.9), eFromV(0.7, 1.3);

V=0.7 E=-16.252352023006083
V=0.7 E=-19.97190015899954
V=0.7 E=-24.3245301623491


In [ ]:
eFromV(0.7, 0.5);

In [ ]:
for v in 0.1:0.1:2.0
    eFromV(v)
end

In [4]:
1 + 1.0I

LoadError: UndefVarError: `I` not defined